In [3]:
import pandas as pd
data = pd.read_csv("Hall_votes.csv")

In [4]:
import numpy as np
rec =[]
for i in range(21104606):
    if (data["congress"][i] == 117):
        rec.append(np.array([data["rollnumber"][i],data["icpsr"][i],data["cast_code"][i]]))

In [5]:
rec = np.array(rec)

In [6]:
lrec = [rec[rec[:, 0] == val] for val in (range(1,997))]

In [7]:
common_values = np.unique(lrec[0][:,1])
for i in range(996):
    common_values = np.intersect1d(np.unique(lrec[i][:,1]),common_values)

In [8]:
common_values= common_values[:-1]

In [9]:
votes = np.zeros((996,411))
for i in range(996):
    for j in range(411):
        a = np.where(lrec[i][:,1] == common_values[j])
        votes[i,j] = lrec[i][a,2]

In [10]:
for i in range(996):
    for j in range(411):
        if votes[i,j] == 7 or votes[i,j] == 9:
            votes[i,j] = np.random.binomial(1, 0.5)
        if (votes[i,j] == 6):
            votes[i,j] = 0

In [16]:
import gurobipy as gp
from gurobipy import GRB

In [17]:
n = 25
m = 51
model = gp.Model('voting')
rep = model.addVars(m,lb=-1.0, ub=1.0, vtype=GRB.CONTINUOUS, name="rep")
bill = model.addVars(n,lb=-1.0, ub=1.0, vtype=GRB.CONTINUOUS, name="bill")
right = model.addVars(n, vtype=GRB.BINARY, name="right")
sign = model.addVars(n,m, vtype=GRB.BINARY, name="sign")

In [18]:
M = 2.01
bill[0] = 0
right[0] = 1
for i in range(n):
    for j in range(m):
        model.addConstr( (-1+2*right[i])*(rep[i]-bill[j]) <= M*sign[i,j]-0.001)
        model.addConstr( (-1+2*right[i])*(bill[i]-rep[j]) <=M*(1-sign[i,j])-0.001)

In [19]:
model.setObjective(sum((sign[i,j]-votes[i,j])**2 for i in range(n) for j in range(m)),GRB.MINIMIZE)
model.write('v.lp')

In [20]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 102 rows, 1376 columns and 204 nonzeros
Model fingerprint: 0x3efd6037
Model has 1275 quadratic objective terms
Model has 2448 quadratic constraints
Variable types: 76 continuous, 1300 integer (1300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [2e+00, 2e+00]
  QLMatrix range   [1e+00, 2e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 2e+00]
  QRHS range       [1e-03, 2e+00]
Presolve added 4896 rows and 0 columns
Presolve removed 0 rows and 2 columns
Presolve time: 0.02s
Presolved: 4998 rows, 1374 columns, 19788 nonzeros
Variable types: 75 continuous, 1299 integer (1299 binary)

Root relaxation: objective 0.000000e+00, 107 ite

In [21]:
print(bill)

{0: 0, 1: <gurobi.Var bill[1] (value -0.9980000000000001)>, 2: <gurobi.Var bill[2] (value 0.0)>, 3: <gurobi.Var bill[3] (value 0.0019880119880111074)>, 4: <gurobi.Var bill[4] (value 0.0)>, 5: <gurobi.Var bill[5] (value 0.0)>, 6: <gurobi.Var bill[6] (value -0.9920000000000008)>, 7: <gurobi.Var bill[7] (value -1.0)>, 8: <gurobi.Var bill[8] (value 0.009988011988010623)>, 9: <gurobi.Var bill[9] (value 0.007988011988010732)>, 10: <gurobi.Var bill[10] (value 0.0)>, 11: <gurobi.Var bill[11] (value 0.0)>, 12: <gurobi.Var bill[12] (value 0.0)>, 13: <gurobi.Var bill[13] (value -0.9920000000000008)>, 14: <gurobi.Var bill[14] (value 0.0)>, 15: <gurobi.Var bill[15] (value 0.00398801198801084)>, 16: <gurobi.Var bill[16] (value 0.9990000000000001)>, 17: <gurobi.Var bill[17] (value 0.0)>, 18: <gurobi.Var bill[18] (value -0.9960000000000003)>, 19: <gurobi.Var bill[19] (value -0.9940000000000005)>, 20: <gurobi.Var bill[20] (value 0.0)>, 21: <gurobi.Var bill[21] (value 0.0)>, 22: <gurobi.Var bill[22] (va

In [22]:
print(rep)

{0: <gurobi.Var rep[0] (value 0.010988011988010513)>, 1: <gurobi.Var rep[1] (value 0.00498801198801073)>, 2: <gurobi.Var rep[2] (value -0.9970000000000002)>, 3: <gurobi.Var rep[3] (value -0.9970000000000002)>, 4: <gurobi.Var rep[4] (value -0.9970000000000002)>, 5: <gurobi.Var rep[5] (value -0.9970000000000002)>, 6: <gurobi.Var rep[6] (value -0.9970000000000002)>, 7: <gurobi.Var rep[7] (value -0.9970000000000002)>, 8: <gurobi.Var rep[8] (value 0.0029880119880109973)>, 9: <gurobi.Var rep[9] (value -0.9970000000000002)>, 10: <gurobi.Var rep[10] (value -0.9970000000000002)>, 11: <gurobi.Var rep[11] (value -0.9990000000000001)>, 12: <gurobi.Var rep[12] (value 0.9980000000000002)>, 13: <gurobi.Var rep[13] (value -0.9970000000000002)>, 14: <gurobi.Var rep[14] (value -0.9970000000000002)>, 15: <gurobi.Var rep[15] (value 1.0)>, 16: <gurobi.Var rep[16] (value -0.9970000000000002)>, 17: <gurobi.Var rep[17] (value 0.010988011988010513)>, 18: <gurobi.Var rep[18] (value -0.9970000000000002)>, 19: <g